In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import tensorflow as tf
import string
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import accuracy_score

from transformers import BertTokenizer, TFBertModel, TFBertForSequenceClassification
from transformers import DistilBertTokenizer, TFDistilBertModel
from transformers import AlbertTokenizer, TFAlbertModel

from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.models import Model

from tqdm import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2024-06-09 22:00:11.253542: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-09 22:00:11.253604: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-09 22:00:11.258491: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-09 22:00:11.419721: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-09 22:00:13.162908: W tensorflow/compiler/

In [2]:
train_df = pd.read_csv("/kaggle/input/spooky-author-identification/train.zip")
test_df = pd.read_csv("/kaggle/input/spooky-author-identification/test.zip")
sub_df = pd.read_csv("/kaggle/input/spooky-author-identification/sample_submission.zip")

In [3]:
texts = train_df['text'].tolist()

In [4]:
"""model_name = 'huawei-noah/TinyBERT_General_4L_312D'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertModel.from_pretrained(model_name, from_pt=True)

def get_embeddings(texts, tokenizer, model, batch_size=32, max_length=512):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Generating Embeddings"):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors='tf', truncation=True, padding='max_length', max_length=max_length)
        outputs = model(inputs)
        cls_embeddings = tf.gather(outputs.last_hidden_state, [0], axis=1)  
        embeddings.append(cls_embeddings)
    return tf.concat(embeddings, axis=0)

embeddings = get_embeddings(texts, tokenizer, model)
embeddings = embeddings.numpy()  

"""

'model_name = \'huawei-noah/TinyBERT_General_4L_312D\'\ntokenizer = BertTokenizer.from_pretrained(model_name)\nmodel = TFBertModel.from_pretrained(model_name, from_pt=True)\n\ndef get_embeddings(texts, tokenizer, model, batch_size=32, max_length=512):\n    embeddings = []\n    for i in tqdm(range(0, len(texts), batch_size), desc="Generating Embeddings"):\n        batch_texts = texts[i:i + batch_size]\n        inputs = tokenizer(batch_texts, return_tensors=\'tf\', truncation=True, padding=\'max_length\', max_length=max_length)\n        outputs = model(inputs)\n        cls_embeddings = tf.gather(outputs.last_hidden_state, [0], axis=1)  \n        embeddings.append(cls_embeddings)\n    return tf.concat(embeddings, axis=0)\n\nembeddings = get_embeddings(texts, tokenizer, model)\nembeddings = embeddings.numpy()  \n\n'

In [5]:
#embeddings = embeddings.reshape(19579,312)

In [6]:
"""
num_clusters = 3
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(embeddings)

labels = kmeans.labels_

data = pd.DataFrame({'text': texts})
data['cluster'] = labels

pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(embeddings)

plt.figure(figsize=(10, 7))
for cluster in range(num_clusters):
    cluster_points = reduced_embeddings[labels == cluster]
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f'Cluster {cluster}')
plt.legend()
plt.title('K-means Clustering of TinyBERT Embeddings (TensorFlow)')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.show()"""

"\nnum_clusters = 3\nkmeans = KMeans(n_clusters=num_clusters, random_state=42)\nkmeans.fit(embeddings)\n\nlabels = kmeans.labels_\n\ndata = pd.DataFrame({'text': texts})\ndata['cluster'] = labels\n\npca = PCA(n_components=2)\nreduced_embeddings = pca.fit_transform(embeddings)\n\nplt.figure(figsize=(10, 7))\nfor cluster in range(num_clusters):\n    cluster_points = reduced_embeddings[labels == cluster]\n    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f'Cluster {cluster}')\nplt.legend()\nplt.title('K-means Clustering of TinyBERT Embeddings (TensorFlow)')\nplt.xlabel('PCA Component 1')\nplt.ylabel('PCA Component 2')\nplt.show()"

In [7]:
# 모델과 토크나이저 로드
model_name = 'huawei-noah/TinyBERT_General_4L_312D'
tokenizer = BertTokenizer.from_pretrained(model_name)
base_model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=3, from_pt=True)

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['fit_denses.0.weight', 'fit_denses.1.weight', 'fit_denses.2.weight', 'fit_denses.1.bias', 'fit_denses.3.weight', 'fit_denses.4.bias', 'fit_denses.3.bias', 'fit_denses.2.bias', 'fit_denses.0.bias', 'fit_denses.4.weight']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you

In [8]:
texts = train_df["text"].tolist()
labels = train_df["author"].tolist()

# 원핫 인코딩
encoder = OneHotEncoder(sparse=False)
labels_encoded = encoder.fit_transform(np.array(labels).reshape(-1, 1))

# 텍스트 인코딩 함수
def encode_texts(texts, tokenizer, max_length=128):
    return tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors='tf')

inputs = encode_texts(texts, tokenizer)
input_ids = inputs['input_ids'].numpy()
attention_masks = inputs['attention_mask'].numpy()

# 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(input_ids, labels_encoded, test_size=0.2, random_state=42)
train_masks, test_masks = train_test_split(attention_masks, test_size=0.2, random_state=42)

# 데이터셋 생성
train_dataset = tf.data.Dataset.from_tensor_slices(((X_train, train_masks), y_train)).batch(8)  # 배치 사이즈 16으로 증가
test_dataset = tf.data.Dataset.from_tensor_slices(((X_test, test_masks), y_test)).batch(8)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [9]:
# 입력 레이어 정의
input_ids_layer = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask_layer = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')

# 람다 레이어를 사용하여 TFBertForSequenceClassification 래핑
def bert_model(inputs):
    input_ids, attention_mask = inputs
    outputs = base_model(input_ids, attention_mask=attention_mask)
    return outputs.logits

logits = tf.keras.layers.Lambda(bert_model, output_shape=(3,))([input_ids_layer, attention_mask_layer])
dense_layer = tf.keras.layers.Dense(3, activation='linear')(logits)
probabilities = tf.keras.layers.Activation('softmax')(dense_layer)


model = tf.keras.Model(inputs=[input_ids_layer, attention_mask_layer], outputs=probabilities)

# 옵티마이저, 손실 함수, 메트릭 정의
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)  # 학습률 조정
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
metric = tf.keras.metrics.CategoricalAccuracy('accuracy')

# 모델 컴파일
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# 모델 학습
history = model.fit(train_dataset, epochs=5, validation_data=test_dataset)  # 에포크 수 증가

# 모델 평가
y_pred = model.predict(test_dataset)
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)
accuracy = accuracy_score(y_test_labels, y_pred_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")

Epoch 1/5
  33/1958 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.4029 - loss: 1.0984

I0000 00:00:1717604427.311923      71 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1717604427.338574      71 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1958/1958 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3912 - loss: 1.0968

W0000 00:00:1717604440.931892      69 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1958/1958 ━━━━━━━━━━━━━━━━━━━━ 24s 9ms/step - accuracy: 0.3912 - loss: 1.0967 - val_accuracy: 0.4009 - val_loss: 1.0924
Epoch 2/5
1958/1958 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.3961 - loss: 1.0924 - val_accuracy: 0.4009 - val_loss: 1.0895
Epoch 3/5
1958/1958 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.3961 - loss: 1.0906 - val_accuracy: 0.4009 - val_loss: 1.0881
Epoch 4/5
1958/1958 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.3961 - loss: 1.0898 - val_accuracy: 0.4009 - val_loss: 1.0875
Epoch 5/5
1958/1958 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.3961 - loss: 1.0895 - val_accuracy: 0.4009 - val_loss: 1.0872
 52/490 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

W0000 00:00:1717604501.208672      68 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


490/490 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step
Accuracy: 40.09%


In [10]:
# 테스트 데이터 예측
test_texts = test_df["text"].tolist()  # 테스트 데이터의 텍스트 컬럼
test_inputs = encode_texts(test_texts, tokenizer)
test_input_ids = test_inputs['input_ids'].numpy()
test_attention_masks = test_inputs['attention_mask'].numpy()

# 테스트 데이터셋 생성
test_dataset = tf.data.Dataset.from_tensor_slices(((test_input_ids, test_attention_masks),)).batch(8)

# 예측 수행
test_pred = model.predict(test_dataset)

sub_df["EAP"] = test_pred[:, 0]
sub_df["HPL"] = test_pred[:, 1]
sub_df["MWS"] = test_pred[:, 2]
# 원핫 인코딩을 디코딩하여 원래 레이블로 변환
#decoded_labels = encoder.inverse_transform(y_test_pred_labels.reshape(-1, 1)).flatten()


# CSV 파일로 저장
sub_df.to_csv('submission.csv', index=False)
print("Submission file saved as submission.csv")

  46/1049 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step

W0000 00:00:1717604518.118883      68 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1049/1049 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step
Submission file saved as submission.csv
